In [26]:
import json
import numpy as np
import pandas as pd
import random
from IPython.display import display

from helper import read_json

In [34]:
def relu(x):
    return np.maximum(0,x)  

# Original features

In [27]:
all_npy = np.load('./npy/all_2372.npy')
movie_genre = np.load('./npy/movie_genre.npy')
usr_following = np.load('./npy/user_followings.npy')
usr_genre = np.load('./npy/user_genre_like.npy')

print('All features:', all_npy.shape)
print('Movie genre:', movie_genre.shape)
print('User following:', usr_following.shape)
print('User genre:', usr_genre.shape)

usr_nb = len(usr_following) # the number of users
movie_nb = len(movie_genre)  # the number of movies
print(usr_nb, movie_nb)

usr_test_amount = 150
movie_test_amount = 32
print(usr_test_amount, movie_test_amount)

latent_dim = 64 # latent dims
ft_dim = all_npy.shape[1] # feature dims
print(latent_dim, ft_dim)

usr_genre_norm = np.zeros(usr_genre.shape)
for i in range(len(usr_genre)):
    usr_genre_norm[i] = usr_genre[i]/np.max(usr_genre[i])
print(usr_genre_norm.shape)
print('Before:', usr_genre)
print('After:', usr_genre_norm)

All features: (165, 2372)
Movie genre: (165, 20)
User following: (1582, 165)
User genre: (1582, 20)
1582 165
150 32
64 2372
(1582, 20)
Before: [[ 2  2  0 ...  1  0  0]
 [34 49 14 ...  1  0  0]
 [10 17 17 ...  4  0  0]
 ...
 [79 43  0 ...  8 12  0]
 [ 6  6  0 ...  0  1  0]
 [67 53  1 ...  3  5  0]]
After: [[0.05263158 0.05263158 0.         ... 0.02631579 0.         0.        ]
 [0.41463415 0.59756098 0.17073171 ... 0.01219512 0.         0.        ]
 [0.24390244 0.41463415 0.41463415 ... 0.09756098 0.         0.        ]
 ...
 [0.30620155 0.16666667 0.         ... 0.03100775 0.04651163 0.        ]
 [0.24       0.24       0.         ... 0.         0.04       0.        ]
 [0.74444444 0.58888889 0.01111111 ... 0.03333333 0.05555556 0.        ]]


In [28]:
usr_idx = [i for i in range(len(usr_following))]
print(len(usr_idx))

random.seed(42)
test_idx = sorted(random.sample(usr_idx, usr_test_amount))
print(len(test_idx), test_idx[:10])
# 150 [13, 51, 54, 61, 65, 88, 93, 96, 114, 130]

1582
150 [13, 51, 54, 61, 65, 88, 93, 96, 114, 130]


In [29]:
with open('../orderedListMovies.json', 'r') as openfile: 
    movie_ls = json.load(openfile)
print(len(movie_ls), movie_ls[:10])

with open('../genresDic.json', 'r') as openfile: 
    genresDic = json.load(openfile)
print(len(genresDic))

165 ['21bridgesmovie', '47metersdown', 'abeautifuldaymovie', 'abominablemovie', 'adastramovie', 'adogsjourneymovie', 'aftermathmovie', 'aftermovie', 'ahiddenlifefilm', 'alitamovie']
166


In [30]:
# Load train/test split
SPLIT_PATH = './split/original_testing/'
train_t = read_json(SPLIT_PATH + 'train_t.json')
train_f = read_json(SPLIT_PATH + 'train_f.json')
test_t = read_json(SPLIT_PATH + 'test_t.json')
test_f = read_json(SPLIT_PATH + 'test_f.json')

# Embedding features

In [31]:
params = np.load('./weight/grid/MRM_E240_33.npz')
U = params['U']
Y = params['Y']
A = params['A']
E = params['E']
Au = params['Wu']
Ay = params['Wy']
Aa = params['Wa']
Av = params['Wv']
B = params['B']

print('User latent shape: ',U.shape)
print('photo latent shape: ', Y.shape)
print('Auxilary latent shape: ',A.shape)
print('Embedding shape:', E.shape)
print('Wu weight shape:', Au.shape)
print('Wy weight shape:', Ay.shape)
print('Wa weight shape:', Aa.shape)
print('Wv weight shape:', Av.shape)
print('Beta shape:',B.shape)

User latent shape:  (1582, 64)
photo latent shape:  (165, 64)
Auxilary latent shape:  (165, 64)
Embedding shape: (240, 2372)
Wu weight shape: (1582, 165, 64)
Wy weight shape: (1582, 165, 64)
Wa weight shape: (1582, 165, 64)
Wv weight shape: (1582, 165, 240)
Beta shape: (1582, 240)


In [32]:
UA = np.dot(U, A.T)
print(UA.shape)
UM = np.dot(U, Y.T)
print(UM.shape)
AM = np.dot(A, Y.T)
print(AM.shape)

(1582, 165)
(1582, 165)
(165, 165)


In [59]:
def testing(U, Y, A, E, Au, Ay, Aa, Av, B):
    #with Embedding
    result = np.zeros((usr_nb, movie_nb))

    #test_idx --> Test 的 index length = 150
    sum_alpha = 0

    for u in range(usr_nb):
        alpha = np.zeros([movie_nb])

        for m in range(movie_nb):
            r = np.max(movie_genre[m] * usr_genre_norm[u]) #sample a 的category vec *user_category vec

            alpha_a = (np.dot(Au[u][m],np.expand_dims(U[u],0).T) + 
                       np.dot(Ay[u][m],np.expand_dims(Y[m],0).T) + 
                       np.dot(Aa[u][m],np.expand_dims(A[m],0).T) +
                       np.dot(Av[u][m],np.dot(E,np.expand_dims(all_npy[m],0).T)))


            # relu part
            alpha[m]=np.sum((relu(alpha_a)))*r

        mul = np.zeros((1,latent_dim))
        added_alpha = np.add(alpha,0.0000000001)
        norm_alpha = added_alpha/np.sum(added_alpha)

        for i in range(movie_nb):
            mul += norm_alpha[i] * A[i] # attention alpha*Ai part

        for k in range(movie_nb):
            result[u][k] = np.dot(mul,Y[k].T) #(U+auxilary)*photo latent factor
            
    print(result.shape)
    return result

In [60]:
result = testing(U, Y, A, E, Au, Ay, Aa, Av, B)

(1582, 165)


In [66]:
topn = 5
for u in range(len(result)):
#     print('\nUser', u)
#     print('=====AM=====')
    desc_A = np.argsort(result[u])[::-1]
    print(u, desc_A[:topn])
#     val = [UA[u][i] for i in desc_A]
#     print(val)
#     movies = [movie_ls[i] for i in desc_A[:topn]]
#     for m in movies:
#         print(m, genresDic[m])

0 [ 86  44 156  58 161]
1 [ 86  44 161 156  58]
2 [ 86  44 156  58 161]
3 [ 86  44 156  58 161]
4 [ 86  44 156  58 119]
5 [ 86  44 156 161  58]
6 [ 86  44  58 119 156]
7 [ 86  44 156 161  58]
8 [ 86  44 161  58 156]
9 [ 86  44 156 161  58]
10 [ 86  44 156  58 161]
11 [ 86  44 156 161  58]
12 [ 86  44 161 156  58]
13 [ 86  44  58 156 119]
14 [ 86  44 161 156  58]
15 [ 86  44 161 156  58]
16 [ 86  44 156 161  58]
17 [ 86 161  44 156  58]
18 [ 86  44 161  58 156]
19 [ 86  44 156  58 161]
20 [ 86  44 156  58 161]
21 [ 86  44 156  58 161]
22 [ 86  44 161  58 156]
23 [ 86  44  58 119 156]
24 [ 86  44 156  58 161]
25 [ 86  44  58 161 156]
26 [ 86  44 161 156  58]
27 [ 86  44 156  58 161]
28 [ 86  44 156 161  58]
29 [ 86  44  58 119 156]
30 [ 86  44 156  58 161]
31 [ 86  44  58 161 156]
32 [ 86  44  58 161 156]
33 [ 86  44 161 156 119]
34 [ 86  44 156 161  58]
35 [ 86  44 161 156  58]
36 [ 86  44 156 161  58]
37 [ 86  44 161 156  58]
38 [ 86  44  58 156 161]
39 [ 86 161  44 156  58]
40 [ 86  4

845 [ 86  44 156  58 161]
846 [ 86  44 161 156  58]
847 [ 86  44 161 156  58]
848 [ 86  44  58 156 161]
849 [ 44  86 119  28  34]
850 [ 86  44  58 156 161]
851 [ 86  44  58 156 161]
852 [ 86  44 156  58 161]
853 [ 86  44 156 161 119]
854 [ 86  44  58 156 161]
855 [ 86  44  58 156 161]
856 [ 86  44 156  58 161]
857 [ 86  44 156  58 161]
858 [ 86  44 156  58 161]
859 [ 86  44 161  58 156]
860 [ 86  44 161 156  58]
861 [ 86  44 161  58 156]
862 [ 86  44 156  58 161]
863 [ 86  44 156  58 161]
864 [ 86  44 161 156  58]
865 [ 86  44  58 156 161]
866 [ 86  44 161 156  58]
867 [ 86  44 161 156  58]
868 [ 86  44  58 161 156]
869 [ 86  44 156 161  58]
870 [ 86  44 161  58 156]
871 [161  40  44  86 102]
872 [ 44  86 119  58 156]
873 [ 86  44 161 156  58]
874 [ 86  44 156 119  58]
875 [ 86 161  44 156  58]
876 [ 86  44 156  58 161]
877 [ 86 161  44 156  58]
878 [ 86  44 156 161 119]
879 [ 86  44 161 156  58]
880 [ 86  44  58 161 156]
881 [ 86  44 161  58 156]
882 [ 44  86 119  58  28]
883 [ 86  44

In [68]:
topn = 5
for u in range(len(UA)):
    print('\nUser', u)
    print('=====follow=====')
#     print('Sum:', sum(usr_following[u]))
    movies = [movie_ls[i] for i in np.nonzero(usr_following[u] == 1)[0]]
    for m in movies:
        print(m, genresDic[m])
    
    desc_A = np.argsort(UA[u])[::-1]
#     print(desc_A[:topn])
#     val = [UA[u][i] for i in desc_A]
#     print(val)
    movies_UA = [movie_ls[i] for i in desc_A[:topn]]
    print('=====UA=====', len(set(movies) & set(movies_UA)))
    for m in movies_UA:
        print(m, genresDic[m])
    
    desc_M = np.argsort(UM[u])[::-1]
#     print(desc_M[:topn])
#     val = [UM[u][i] for i in desc_M]
#     print(val)
    movies_UM = [movie_ls[i] for i in desc_M[:topn]]
    print('=====UM=====', len(set(movies) & set(movies_UM)))
    for m in movies_UM:
        print(m, genresDic[m])


User 0
=====follow=====
abeautifuldaymovie ['Biography', 'Drama']
captivestatemovie ['Drama', 'Sci-Fi', 'Thriller']
coldpursuitmovie ['Action', 'Crime', 'Drama']
currentwarmovie ['Biography', 'Drama', 'History']
fordvferrari ['Action', 'Biography', 'Drama']
goodliarmovie ['Drama']
longshotmovie ['Comedy', 'Romance']
peanutbutterfalcon ['Adventure', 'Comedy', 'Drama']
richardjewellfilm ['Drama']
thebestofenemies ['Biography', 'Drama', 'History']
=====UA===== 0
thehummingbirdprojectfilm ['Thriller']
angrybirdsmovie ['Adventure', 'Animation', 'Comedy']
spidermanmovie ['Action', 'Adventure', 'Sci-Fi']
godzillamovie ['Action', 'Adventure', 'Fantasy']
thekitchenmovie ['Action', 'Crime', 'Drama']
=====UM===== 0
lordsofchaosmovie ['Biography', 'Drama', 'Horror']
yardiefilm ['Crime', 'Drama']
aftermovie ['Drama', 'Romance']
piercingmovie ['Horror', 'Mystery', 'Thriller']
thehighwaymen ['Biography', 'Crime', 'Drama']

User 1
=====follow=====
abominablemovie ['Adventure', 'Animation', 'Comedy']


=====UA===== 1
glassmovie ['Drama', 'Sci-Fi', 'Thriller']
thebestofenemies ['Biography', 'Drama', 'History']
triplefrontier ['Action', 'Adventure', 'Crime']
jumanjimovie ['Action', 'Adventure', 'Comedy']
countdown ['Horror', 'Thriller']
=====UM===== 1
brightburnmovie ['Horror', 'Sci-Fi']
knivesout ['Comedy', 'Crime', 'Drama']
doctorsleepmovie ['Horror']
xmenmovies ['Action', 'Adventure', 'Sci-Fi']
glassmovie ['Drama', 'Sci-Fi', 'Thriller']

User 56
=====follow=====
alitamovie ['Action', 'Adventure', 'Sci-Fi']
booksmart ['Comedy']
childsplaymovie ['Horror']
doctorsleepmovie ['Horror']
hellboymovie ['Action', 'Adventure', 'Fantasy']
jayandsilentbob ['Comedy']
lalloronamovie ['Horror', 'Mystery', 'Thriller']
longshotmovie ['Comedy', 'Romance']
scarystoriesmovie ['Horror', 'Mystery', 'Thriller']
theprodigymovie ['Horror', 'Thriller']
xmenmovies ['Action', 'Adventure', 'Sci-Fi']
=====UA===== 0
hersmellmovie ['Drama', 'Music']
artofracingmovie ['Comedy', 'Drama', 'Romance']
rocketmanmovie ['

thedeaddontdie ['Comedy', 'Fantasy', 'Horror']
thenightingalefilm ['Adventure', 'Drama', 'Thriller']
=====UA===== 0
theirishmanfilm ['Biography', 'Crime', 'Drama']
fastandfuriousmovie ['Action', 'Adventure']
teenspiritmov ['Drama', 'Music']
abominablemovie ['Adventure', 'Animation', 'Comedy']
johnwickmovie ['Action', 'Crime', 'Thriller']
=====UM===== 1
richardjewellfilm ['Drama']
blackandbluemovie ['Action', 'Crime', 'Drama']
peanutbutterfalcon ['Adventure', 'Comedy', 'Drama']
dontletgomovie ['Drama', 'Fantasy', 'Horror']
gretafilm ['Drama', 'Mystery', 'Thriller']

User 110
=====follow=====
adogsjourneymovie ['Adventure', 'Comedy', 'Drama']
alitamovie ['Action', 'Adventure', 'Sci-Fi']
breakthroughmovie ['Biography', 'Drama']
captainmarvelofficial ['Action', 'Adventure', 'Sci-Fi']
fightingwmyfam ['Biography', 'Comedy', 'Drama']
gloriabellmovie ['Comedy', 'Drama', 'Romance']
httydragon ['Action', 'Adventure', 'Animation']
isntitromantic ['Comedy', 'Fantasy', 'Romance']
johnwickmovie ['Ac

lordsofchaosmovie ['Biography', 'Drama', 'Horror']
thehighwaymen ['Biography', 'Crime', 'Drama']

User 165
=====follow=====
21bridgesmovie ['Action', 'Crime', 'Drama']
abominablemovie ['Adventure', 'Animation', 'Comedy']
adastramovie ['Adventure', 'Drama', 'Mystery']
aftermathmovie ['Drama', 'Romance', 'War']
alitamovie ['Action', 'Adventure', 'Sci-Fi']
angelhasfallen ['Action', 'Thriller']
annabellemovie ['Horror', 'Mystery', 'Thriller']
annamovie ['Action', 'Thriller']
artofracingmovie ['Comedy', 'Drama', 'Romance']
blackandbluemovie ['Action', 'Crime', 'Drama']
blackchristmas ['Horror', 'Mystery', 'Thriller']
blindedbythelightmovie ['Comedy', 'Drama', 'Music']
bombshellmovie ['Biography', 'Drama']
booksmart ['Comedy']
breakthroughmovie ['Biography', 'Drama']
captainmarvelofficial ['Action', 'Adventure', 'Sci-Fi']
captivestatemovie ['Drama', 'Sci-Fi', 'Thriller']
coldpursuitmovie ['Action', 'Crime', 'Drama']
countdown ['Horror', 'Thriller']
darkwatersmovie ['Biography', 'Drama', 'His

=====UA===== 0
detectivepikachumovie ['Action', 'Adventure', 'Comedy']
downtonabbey_official ['Drama']
uglydolls ['Adventure', 'Animation', 'Comedy']
lastblackmansf ['Drama']
judythefilm ['Biography', 'Drama', 'History']
=====UM===== 2
meninblack ['Action', 'Adventure', 'Comedy']
glassmovie ['Drama', 'Sci-Fi', 'Thriller']
midwaymovie ['Action', 'Drama', 'History']
knivesout ['Comedy', 'Crime', 'Drama']
usmovie ['Horror', 'Mystery', 'Thriller']

User 218
=====follow=====
21bridgesmovie ['Action', 'Crime', 'Drama']
alitamovie ['Action', 'Adventure', 'Sci-Fi']
angelhasfallen ['Action', 'Thriller']
annamovie ['Action', 'Thriller']
avengers ['Action', 'Adventure', 'Sci-Fi']
blackandbluemovie ['Action', 'Crime', 'Drama']
blackchristmas ['Horror', 'Mystery', 'Thriller']
bombshellmovie ['Biography', 'Drama']
brightburnmovie ['Horror', 'Sci-Fi']
brittanyrunsmov ['Comedy', 'Drama']
coldpursuitmovie ['Action', 'Crime', 'Drama']
doctorsleepmovie ['Horror']
dolemiteismyname ['Biography', 'Comedy', 

...

=====UM===== 2
usmovie ['Horror', 'Mystery', 'Thriller']
meninblack ['Action', 'Adventure', 'Comedy']
doctorsleepmovie ['Horror']
happydeathdaymovie ['Comedy', 'Horror', 'Mystery']
thelegomovie ['Action', 'Adventure', 'Animation']

User 1363
=====follow=====
abeautifuldaymovie ['Biography', 'Drama']
adastramovie ['Adventure', 'Drama', 'Mystery']
ahiddenlifefilm ['Biography', 'Drama', 'War']
apollo11movie ['Documentary', 'History']
bombshellmovie ['Biography', 'Drama']
darkwatersmovie ['Biography', 'Drama', 'History']
dolemiteismyname ['Biography', 'Comedy', 'Drama']
fordvferrari ['Action', 'Biography', 'Drama']
harrietfilm ['Biography', 'Drama', 'History']
hersmellmovie ['Drama', 'Music']
highlifemovie ['Adventure', 'Drama', 'Mystery']
honeyboymovie ['Drama']
hustlersmovie ['Comedy', 'Crime', 'Drama']
jojorabbitmovie ['Comedy', 'Drama', 'War']
jokermovie ['Crime', 'Drama', 'Thriller']
judythefilm ['Biography', 'Drama', 'History']
justmercyfilm ['Drama']
knivesout ['Comedy', 'Crime', 'D

piercingmovie ['Horror', 'Mystery', 'Thriller']

User 1415
=====follow=====
blindedbythelightmovie ['Comedy', 'Drama', 'Music']
brightburnmovie ['Horror', 'Sci-Fi']
captivestatemovie ['Drama', 'Sci-Fi', 'Thriller']
dumbo ['Adventure', 'Family', 'Fantasy']
fightingwmyfam ['Biography', 'Comedy', 'Drama']
fordvferrari ['Action', 'Biography', 'Drama']
glassmovie ['Drama', 'Sci-Fi', 'Thriller']
hellboymovie ['Action', 'Adventure', 'Fantasy']
longshotmovie ['Comedy', 'Romance']
midwaymovie ['Action', 'Drama', 'History']
missinglinkfilm ['Adventure', 'Animation', 'Comedy']
peanutbutterfalcon ['Adventure', 'Comedy', 'Drama']
shaftmovie ['Action', 'Comedy', 'Crime']
thedeaddontdie ['Comedy', 'Fantasy', 'Horror']
thelegomovie ['Action', 'Adventure', 'Animation']
uncutgems ['Comedy', 'Crime', 'Drama']
=====UA===== 0
happydeathdaymovie ['Comedy', 'Horror', 'Mystery']
thebestofenemies ['Biography', 'Drama', 'History']
meninblack ['Action', 'Adventure', 'Comedy']
jokermovie ['Crime', 'Drama', 'Thril

=====UA===== 0
honeyboymovie ['Drama']
adastramovie ['Adventure', 'Drama', 'Mystery']
hersmellmovie ['Drama', 'Music']
apollo11movie ['Documentary', 'History']
catsmovie ['Animation', 'Comedy', 'Drama']
=====UM===== 1
dontletgomovie ['Drama', 'Fantasy', 'Horror']
brightburnmovie ['Horror', 'Sci-Fi']
readyornotfilm ['Comedy', 'Horror', 'Mystery']
doctorsleepmovie ['Horror']
childsplaymovie ['Horror']

User 1468
=====follow=====
adogsjourneymovie ['Adventure', 'Comedy', 'Drama']
artofracingmovie ['Comedy', 'Drama', 'Romance']
dumbo ['Adventure', 'Family', 'Fantasy']
godzillamovie ['Action', 'Adventure', 'Fantasy']
happydeathdaymovie ['Comedy', 'Horror', 'Mystery']
hellboymovie ['Action', 'Adventure', 'Fantasy']
hustlemovie ['Comedy', 'Crime']
littlethemovie ['Comedy', 'Fantasy', 'Romance']
longshotmovie ['Comedy', 'Romance']
stubermovie ['Action', 'Comedy', 'Crime']
theintrudermovie ['Drama', 'Horror', 'Mystery']
thesunisalsoastar ['Drama', 'Romance']
whatmenwant ['Comedy', 'Fantasy', 'R

=====UM===== 0
yardiefilm ['Crime', 'Drama']
lordsofchaosmovie ['Biography', 'Drama', 'Horror']
aftermovie ['Drama', 'Romance']
wrinklestheclown ['Comedy', 'Documentary', 'Horror']
brittanyrunsmov ['Comedy', 'Drama']

User 1522
=====follow=====
abominablemovie ['Adventure', 'Animation', 'Comedy']
catsmovie ['Animation', 'Comedy', 'Drama']
disneyfrozen ['Adventure', 'Animation', 'Comedy']
doctorsleepmovie ['Horror']
itmovieofficial ['Drama', 'Fantasy', 'Horror']
jokermovie ['Crime', 'Drama', 'Thriller']
lionking ['Adventure', 'Animation', 'Drama']
meettheaddams ['Animation', 'Comedy', 'Family']
playingwithfire ['Comedy']
playmobilthemovie ['Adventure', 'Animation', 'Comedy']
spiesindisguise ['Action', 'Adventure', 'Animation']
toystory ['Adventure', 'Animation', 'Comedy']
uglydolls ['Adventure', 'Animation', 'Comedy']
=====UA===== 0
lastblackmansf ['Drama']
terminator ['Action', 'Adventure', 'Sci-Fi']
bernadettefilm ['Comedy', 'Drama', 'Mystery']
knivesout ['Comedy', 'Crime', 'Drama']
x

=====UM===== 0
knivesout ['Comedy', 'Crime', 'Drama']
xmenmovies ['Action', 'Adventure', 'Sci-Fi']
doctorsleepmovie ['Horror']
usmovie ['Horror', 'Mystery', 'Thriller']
petsematarymovie ['Horror', 'Mystery', 'Thriller']

User 1575
=====follow=====
21bridgesmovie ['Action', 'Crime', 'Drama']
47metersdown ['Adventure', 'Drama', 'Horror']
abominablemovie ['Adventure', 'Animation', 'Comedy']
adastramovie ['Adventure', 'Drama', 'Mystery']
angelhasfallen ['Action', 'Thriller']
artofracingmovie ['Comedy', 'Drama', 'Romance']
charliesangels ['Action', 'Adventure', 'Comedy']
geminimanmovie ['Action', 'Drama', 'Sci-Fi']
godzillamovie ['Action', 'Adventure', 'Fantasy']
hellboymovie ['Action', 'Adventure', 'Fantasy']
meninblack ['Action', 'Adventure', 'Comedy']
midwaymovie ['Action', 'Drama', 'History']
onceinhollywood ['Comedy', 'Drama']
peanutbutterfalcon ['Adventure', 'Comedy', 'Drama']
rambomovie ['Action', 'Adventure', 'Thriller']
rocketmanmovie ['Biography', 'Drama', 'Music']
terminator ['Ac